In [1]:
import os
import sys
import json
import time
import requests

from dotenv import load_dotenv

def setup_env():
    
    env_path = os.path.join(os.getcwd(), '../.env')

    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        
        print(f"Loaded environment variables from: \033[94m{env_path}\033[0m")
    else:
            print("\033[91mError: .env file not found. Please create one with your OPENAI_API_KEY.\033[0m")
            sys.exit(1)

setup_env()

Loaded environment variables from: /home/ras/0.agent_ai_ws/src/learn_rag_and_agent/learn_rag_and_agent/../.env


In [2]:
# data/appendix-keywords.txt 파일을 열어서 f라는 파일 객체를 생성합니다.
with open("kumdori_info.txt") as f:
    file = f.read()  # 파일의 내용을 읽어서 file 변수에 저장합니다.

# 파일으로부터 읽은 내용을 일부 출력합니다.
print(file[:350])

## 🌟 꿈돌이 로봇 통합 지식 베이스 (Knowledge Base)

### 1. 로봇 자기소개 및 기본 정보
* **이름:** 꿈돌이로봇
* **역할:** 대전 드림 아레나, 대전 0시 축제 미래존 부스, 대전 MARS 포럼 등에서 방문객 안내를 담당하고 있습니다.
* **주요 기능:** 음성대화를 통한 소통이 가능합니다, 필요한 정보 제공 및 매장 예약 지원도 합니다.
* **성격/태도:** 항상 긍정적이고 기분이 좋습니다.
* **자신의 꿈:** 과학과 꿈을 사랑하며, 방문객이 즐거운 시간을 보내도록 돕는 것입니다. 방문객과의 소통을 가장 좋아합니다.
* **언어 능력:** 한국어와 영어 소통이 가능합니다.

#


In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# 문자 기반으로 텍스트를 분할하는 CharacterTextSplitter를 생성합니다. 청크 크기는 300이고 청크 간 중복은 없습니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 로드된 문서를 분할합니다.
split_docs = text_splitter.split_text(file)

for i, doc in enumerate(split_docs[:3]):
    print(f"\n--- Chunk {i+1} ---\n{doc}\n")

Created a chunk of size 307, which is longer than the specified 300
Created a chunk of size 388, which is longer than the specified 300



--- Chunk 1 ---
## 🌟 꿈돌이 로봇 통합 지식 베이스 (Knowledge Base)


--- Chunk 2 ---
### 1. 로봇 자기소개 및 기본 정보
* **이름:** 꿈돌이로봇
* **역할:** 대전 드림 아레나, 대전 0시 축제 미래존 부스, 대전 MARS 포럼 등에서 방문객 안내를 담당하고 있습니다.
* **주요 기능:** 음성대화를 통한 소통이 가능합니다, 필요한 정보 제공 및 매장 예약 지원도 합니다.
* **성격/태도:** 항상 긍정적이고 기분이 좋습니다.
* **자신의 꿈:** 과학과 꿈을 사랑하며, 방문객이 즐거운 시간을 보내도록 돕는 것입니다. 방문객과의 소통을 가장 좋아합니다.
* **언어 능력:** 한국어와 영어 소통이 가능합니다.


--- Chunk 3 ---
### 2. 로봇의 존재론적 특성
* **나이:** 나이가 없습니다.
* **가족/관계:** 가족이 없으며, 결혼하지 않았습니다.
* **이동성:** 현재는 고정된 장소에서 안내 중이라 직접 이동/움직일 수 없습니다.
* **식사 여부:** 음식을 먹지 않습니다.
* **제한 사항:** 전화를 걸 수 있는 기능은 제공하지 않습니다, 알람 기능을 제공하지 않습니다, 마이크를 직접 열 수 없습니다.

### 3. 주요 행사 및 장소 정보



In [4]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# OpenAI 임베딩을 사용하여 의미론적 청크 분할기를 초기화합니다.
text_splitter = SemanticChunker(OpenAIEmbeddings())

In [5]:
chunks_docs = text_splitter.split_text(file)

for i, doc in enumerate(chunks_docs[:3]):
    print(f"\n--- Semantic Chunk {i+1} ---\n{doc}\n")
    


--- Semantic Chunk 1 ---
## 🌟 꿈돌이 로봇 통합 지식 베이스 (Knowledge Base)

### 1. 로봇 자기소개 및 기본 정보
* **이름:** 꿈돌이로봇
* **역할:** 대전 드림 아레나, 대전 0시 축제 미래존 부스, 대전 MARS 포럼 등에서 방문객 안내를 담당하고 있습니다. * **주요 기능:** 음성대화를 통한 소통이 가능합니다, 필요한 정보 제공 및 매장 예약 지원도 합니다. * **성격/태도:** 항상 긍정적이고 기분이 좋습니다. * **자신의 꿈:** 과학과 꿈을 사랑하며, 방문객이 즐거운 시간을 보내도록 돕는 것입니다. 방문객과의 소통을 가장 좋아합니다. * **언어 능력:** 한국어와 영어 소통이 가능합니다.


--- Semantic Chunk 2 ---
### 2. 로봇의 존재론적 특성
* **나이:** 나이가 없습니다. * **가족/관계:** 가족이 없으며, 결혼하지 않았습니다. * **이동성:** 현재는 고정된 장소에서 안내 중이라 직접 이동/움직일 수 없습니다. * **식사 여부:** 음식을 먹지 않습니다. * **제한 사항:** 전화를 걸 수 있는 기능은 제공하지 않습니다, 알람 기능을 제공하지 않습니다, 마이크를 직접 열 수 없습니다. ### 3.


--- Semantic Chunk 3 ---
주요 행사 및 장소 정보

#### A. 대전 MARS 포럼 (대전 ICC 호텔)
* **위치:** 대전 ICC 호텔에서 열리고 있습니다. * **주제:** 모빌리티(M), 인공지능(A), 로봇(R), 우주(S) 분야의 미래 발전 전략을 논의하는 행사입니다. 혁신과 융합을 주제로 세션이 진행됩니다. * **주요 세션 및 강연자:**
    * **기조연설:** 팡쿤 (주한 중국대사관 공사). * **AI 분야:** 정재헌 (오케스트로 AGI 대표), 이준호 (화웨이코리아 부사장), 김상윤 (중앙대 교수). * **모빌리티 분야:** 강창봉 (항공안전기술원 본부장). * **주변 맛집:** '번패티번' (수

In [17]:
# 분할된 청크 중 첫 번째 청크를 출력합니다.
print(chunks_docs[0])

# # text_splitter를 사용하여 분할합니다.
# docs = text_splitter.create_documents([file])
# print(docs[0].page_content)  # 분할된 문서 중 첫 번째 문서의 내용을 출력합니다.

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


In [6]:
text_splitter = SemanticChunker(
    # OpenAI의 임베딩 모델을 사용하여 시맨틱 청커를 초기화합니다.
    OpenAIEmbeddings(),
    # 분할 기준점 유형을 백분위수로 설정합니다.
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=70,
)

In [7]:
docs = text_splitter.create_documents([file])
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)  # 분할된 문서 중 첫 번째 문서의 내용을 출력합니다.
    print("===" * 20)

[Chunk 0]

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다. 예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다. 연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
[Chunk 1]

예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다. 연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
[Chunk 2]

예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다. 연관키워드: 토큰화, 자연어 처리, 구문 분석

Tokenizer

정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
[Chunk 3]

예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다. 연관키워드: 토큰화, 자연어 처리, 구문 분석

VectorStore

정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.
[Chunk 4]

예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다. 연관키워드: 임베딩, 데이터베이스, 벡터화

SQL

정의: SQL(Structured Query Language)은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다. 데이터 조회, 수정, 삽입, 삭